<a href="https://colab.research.google.com/github/yamidur/POO-2025_2/blob/main/Actividad_6_POO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Actividad 6


## Ejercicio 8.4 pág. 517
## Cuadros de diálogo

In [ ]:
# @title
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
import os
import pandas as pd
from enum import Enum


# 1. Definición de Clases POO


# TipoCargo y TipoGénero
class TipoCargo(Enum):
    DIRECTIVO = "Directivo"
    ESTRATEGICO = "Estratégico"
    OPERATIVO = "Operativo"

class TipoGenero(Enum):
    MASCULINO = "Masculino"
    FEMENINO = "Femenino"

class Empleado:
    """
    Clase que modela un empleado de una empresa, adaptada del ejercicio 8.4 de Java.
    """
    def __init__(self, nombre, apellidos, cargo, genero, salario_dia,
                 dias_trabajados, otros_ingresos, pagos_salud, aporte_pensiones):

        self.nombre = nombre
        self.apellidos = apellidos
        self.cargo = cargo # Instancia de TipoCargo
        self.genero = genero # Instancia de TipoGenero
        self.salario_dia = salario_dia
        self.dias_trabajados = dias_trabajados
        self.otros_ingresos = otros_ingresos
        self.pagos_salud = pagos_salud
        self.aporte_pensiones = aporte_pensiones

    def calcular_nomina(self):
        """
        Calcula el salario mensual del empleado.
        Salario mensual = (días trabajados * sueldo por día) + otros ingresos -
                          pagos por salud - aporte pensiones
        """
        salario_base = self.salario_dia * self.dias_trabajados
        salario_mensual = salario_base + self.otros_ingresos - self.pagos_salud - self.aporte_pensiones
        return salario_mensual

    def __str__(self):
        """Método para obtener una representación en texto del empleado."""
        nomina = self.calcular_nomina()
        return (
            f"Nombre = {self.nombre}\n"
            f"Apellidos = {self.apellidos}\n"
            f"Cargo = {self.cargo.value}\n"
            f"Género = {self.genero.value}\n"
            f"Salario por día = ${self.salario_dia:.2f}\n"
            f"Días trabajados = {self.dias_trabajados}\n"
            f"Otros ingresos = ${self.otros_ingresos:.2f}\n"
            f"Pagos por salud = ${self.pagos_salud:.2f}\n"
            f"Aportes pensiones = ${self.aporte_pensiones:.2f}\n"
            f"Salario Mensual = ${nomina:.2f}\n"
            f"----------------------------------------"
        )


class ListaEmpleados:
    """
    Clase que maneja la lista de objetos Empleado y calcula la nómina total.
    """
    def __init__(self):
        self.lista = []

    def agregar_empleado(self, empleado):
        """Agrega un objeto Empleado a la lista."""
        self.lista.append(empleado)

    def calcular_total_nomina(self):
        """Calcula el total de la nómina mensual de la empresa."""
        total = sum(e.calcular_nomina() for e in self.lista)
        return total

    def obtener_matriz(self):
        """Convierte la lista de empleados a una estructura de datos para la tabla."""
        datos = []
        for e in self.lista:
            datos.append({
                'NOMBRE': e.nombre,
                'APELLIDOS': e.apellidos,
                'SUELDO ($)': f"{e.calcular_nomina():.2f}"
            })
        return pd.DataFrame(datos)

    def convertir_texto(self):
        """Convierte los datos de la lista de empleados a un string para guardar en archivo."""
        texto = ""
        for e in self.lista:
            texto += str(e) + "\n"

        total_nomina = self.calcular_total_nomina()
        texto += f"\nTotal nómina de la empresa = ${total_nomina:.2f}"
        return texto


# 2. Lógica de la Interfaz Gráfica (ipywidgets)


# INSTANCIA PRINCIPAL
empleados_db = ListaEmpleados()

# --- Componentes Comunes ---
output_log = widgets.Output() # Para mensajes de error o confirmación
output_nomina = widgets.Output() # Para mostrar la tabla de nómina

# --- Agregar Empleado (Formulario) ---

# Etiquetas y campos de entrada
campo_nombre = widgets.Text(description='Nombre:', placeholder='Nombre')
campo_apellidos = widgets.Text(description='Apellidos:', placeholder='Apellidos')

# Cargo
opciones_cargo = [c.value for c in TipoCargo]
campo_cargo = widgets.Dropdown(description='Cargo:', options=opciones_cargo, value=TipoCargo.OPERATIVO.value)

# Género
grupo_genero = widgets.RadioButtons(description='Género:', options=[g.value for g in TipoGenero], value=TipoGenero.MASCULINO.value)

# Salario y otros valores (Todos Double)
campo_salario_dia = widgets.FloatText(description='Salario/Día ($):', value=0.0, step=0.01,layout=widgets.Layout(width='450px'),style={'description_width': '135px'})
campo_otros_ingresos = widgets.FloatText(description='Otros Ingresos ($):', value=0.0, step=0.01,layout=widgets.Layout(width='450px'),style={'description_width': '135px'})
campo_pagos_salud = widgets.FloatText(description='Pagos Salud ($):', value=0.0, step=0.01,layout=widgets.Layout(width='450px'),style={'description_width': '135px'})
campo_pensiones = widgets.FloatText(description='Aporte Pensiones ($):', value=0.0, step=0.01,layout=widgets.Layout(width='450px'),style={'description_width': '135px'})

# Días trabajados
campo_dias_trabajados = widgets.IntSlider(
    description='Días Trabajados:',
    min=1, max=31, value=30,layout=widgets.Layout(width='450px'),style={'description_width': '135px'},
    continuous_update=False
)

# Botones
btn_agregar = widgets.Button(description="Agregar Empleado", button_style='success')
btn_limpiar_form = widgets.Button(description="Limpiar Formulario", button_style='info')

# Diseño del Formulario de Agregar Empleado
form_agregar_empleado = widgets.VBox([
    widgets.Label("### 1. Agregar Empleado"),
    campo_nombre,
    campo_apellidos,
    campo_cargo,
    grupo_genero,
    campo_salario_dia,
    campo_dias_trabajados,
    campo_otros_ingresos,
    campo_pagos_salud,
    campo_pensiones,
    widgets.HBox([btn_agregar, btn_limpiar_form])
])

# --- Lógica de Agregar Empleado ---

def limpiar_campos_formulario(b=None):
    """Limpia todos los campos del formulario."""
    campo_nombre.value = ''
    campo_apellidos.value = ''
    campo_cargo.value = TipoCargo.OPERATIVO.value
    grupo_genero.value = TipoGenero.MASCULINO.value
    campo_salario_dia.value = 0.0
    campo_dias_trabajados.value = 30
    campo_otros_ingresos.value = 0.0
    campo_pagos_salud.value = 0.0
    campo_pensiones.value = 0.0
    with output_log:
        clear_output(wait=True)
        print(">> Formulario limpiado.")

def agregar_empleado_action(b):
    """Valida los datos y añade un nuevo Empleado a la lista."""
    with output_log:
        clear_output(wait=True)

        # Validación de campos obligatorios
        if not campo_nombre.value or not campo_apellidos.value:
            print("ERROR: Nombre y Apellidos son obligatorios.")
            return

        try:
            # Recolección y conversión de datos
            nombre = campo_nombre.value.strip()
            apellidos = campo_apellidos.value.strip()

            # Mapeo de valores
            tipo_c = TipoCargo(campo_cargo.value)
            tipo_g = TipoGenero(grupo_genero.value)

            salario_dia = campo_salario_dia.value
            dias_trabajados = campo_dias_trabajados.value
            otros_ingresos = campo_otros_ingresos.value
            pagos_salud = campo_pagos_salud.value
            aporte_pensiones = campo_pensiones.value

            # Crear y agregar el empleado
            empleado = Empleado(nombre, apellidos, tipo_c, tipo_g,
                                salario_dia, dias_trabajados,
                                otros_ingresos, pagos_salud, aporte_pensiones)

            empleados_db.agregar_empleado(empleado)

            print(f"ÉXITO: Empleado '{nombre} {apellidos}' agregado.")
            limpiar_campos_formulario()

        except Exception as e:
            print(f"ERROR en el formato de datos: {e}")
            print("Asegúrese de que todos los valores numéricos sean válidos.")

# Conexión de botones
btn_agregar.on_click(agregar_empleado_action)
btn_limpiar_form.on_click(limpiar_campos_formulario)


# --- Calcular Nómina (Tabla) ---

btn_calcular_nomina = widgets.Button(description="Calcular Nómina", button_style='primary')

def calcular_nomina_action(b):
    """Muestra la tabla de empleados y el total de la nómina."""
    with output_nomina:
        clear_output(wait=True)

        if not empleados_db.lista:
            display(HTML("<p style='color: orange;'>No hay empleados ingresados para calcular la nómina.</p>"))
            return

        # 1. Obtener la tabla (usando pandas DataFrame)
        df = empleados_db.obtener_matriz()

        # 2. Calcular el total
        total_nomina = empleados_db.calcular_total_nomina()

        # 3. Mostrar la tabla
        display(HTML("### Nómina de Empleados"))
        display(df) # Muestra la tabla de DataFrame

        # 4. Mostrar el total en la parte inferior
        total_html = f"<h4 style='text-align: right; color: #007bff;'>Total Nómina de la Empresa: ${total_nomina:,.2f}</h4>"
        display(HTML(total_html))

btn_calcular_nomina.on_click(calcular_nomina_action)

# ---  Guardar Archivo ---

btn_guardar_archivo = widgets.Button(description="Guardar Archivo (Nómina.txt)", button_style='danger')

def guardar_archivo_action(b):
    """Guarda los datos de la nómina en un archivo Nómina.txt en el directorio de Colab."""
    file_path = "Nómina.txt"

    with output_log:
        clear_output(wait=True)

        if not empleados_db.lista:
            print("ADVERTENCIA: No hay datos para guardar.")
            return

        try:
            contenido = empleados_db.convertir_texto()

            # Guardar el archivo
            with open(file_path, "w", encoding="utf-8") as f:
                f.write(contenido)

            # Mensaje de confirmación
            print(f"ÉXITO: El archivo de la nómina '{file_path}' se ha creado en el directorio actual de Colab.")

        except Exception as e:
            print(f"ERROR: No se pudo escribir el archivo. {e}")

btn_guardar_archivo.on_click(guardar_archivo_action)



# 3. Estructura y Display de la Aplicación Principal


# Se crea un contenedor de pestañas para simular los menús/ventanas
tab_widget = widgets.Tab()
tab_widget.children = [
    form_agregar_empleado,
    widgets.VBox([btn_calcular_nomina, output_nomina]),
    widgets.VBox([btn_guardar_archivo, output_log])
]

# Títulos de las pestañas
tab_widget.set_title(0, 'Agregar Empleado')
tab_widget.set_title(1, 'Calcular Nómina')
tab_widget.set_title(2, 'Guardar Archivo')

# Título Principal
titulo = widgets.HTML("<h2>Sistema de Nómina (Ejercicio 8.4)</h2>")

# Diseño final de la aplicación
app_layout = widgets.VBox([
    titulo,
    tab_widget,
    widgets.HTML("<hr>"),
    widgets.Label("Log de Operaciones y Mensajes:"),
    output_log
])

# Mostrar la interfaz en Colab
display(app_layout)

## Ejercicio 8.5 página 546
## Gestión de contenidos

In [ ]:
# @title
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
from datetime import datetime
import pandas as pd
from IPython.display import HTML
display(HTML("""
<style>
.widget-label {
    min-width: 200px !important;
    width: auto !important;
    white-space: normal !important;
}

/* Permite botones más anchos automáticamente */
button.jupyter-widgets.jupyter-button {
    width: auto !important;
    max-width: 20% !important;
}
</style>
"""))

# Constantes de Precios
PRECIO_SIMPLE = 120000
PRECIO_PREMIUM = 160000
NUM_HABITACIONES = 10

# Configuración de Layout para etiquetas
WIDGET_LAYOUT = widgets.Layout(width='600px', description_width='350px')
TINY_LAYOUT = widgets.Layout(width='300px', description_width='150px')

# ==============================================================================
# 1. Clases POO del Enunciado
# ==============================================================================

class Huesped:
    """Clase que representa a la persona que ocupa la habitación."""
    def __init__(self, nombre, apellidos, documento):
        self.nombre = nombre
        self.apellidos = apellidos
        self.documento = documento

    def __str__(self):
        return f"{self.nombre} {self.apellidos} (ID: {self.documento})"

class Habitacion:
    """Clase que modela una habitación de hotel."""
    def __init__(self, numero):
        self.numero = numero
        self.precio_dia = PRECIO_SIMPLE if numero <= 5 else PRECIO_PREMIUM
        self.disponible = True
        self.huesped = None       # Objeto Huesped cuando está ocupada
        self.fecha_ingreso = None # Objeto datetime.date o None

    def ocupar(self, huesped, fecha_ingreso):
        """Marca la habitación como ocupada y asigna huésped y fecha de ingreso."""
        self.disponible = False
        self.huesped = huesped
        self.fecha_ingreso = fecha_ingreso

    def liberar(self):
        """Libera la habitación y limpia los datos del huésped."""
        self.disponible = True
        self.huesped = None
        self.fecha_ingreso = None

    def get_estado(self):
        """Retorna el estado de la habitación en texto."""
        return "Disponible" if self.disponible else f"No disponible (Huésped: {self.huesped.nombre} - Ingreso: {self.fecha_ingreso.strftime('%d/%m/%Y')})"

class Hotel:
    """Clase principal que contiene y gestiona todas las habitaciones."""
    def __init__(self):
        # Crea las 10 habitaciones
        self.habitaciones = [Habitacion(i) for i in range(1, NUM_HABITACIONES + 1)]

    def get_habitacion(self, numero):
        """Retorna el objeto Habitacion por su número (1 a 10)."""
        if 1 <= numero <= NUM_HABITACIONES:
            # La lista es base 0, el número de habitación es base 1
            return self.habitaciones[numero - 1]
        return None

    def get_listado_df(self):
        """Retorna un DataFrame con el estado de todas las habitaciones."""
        data = []
        for hab in self.habitaciones:
            data.append({
                'Habitación': hab.numero,
                'Precio/Día ($)': f"{hab.precio_dia:,.0f}",
                'Estado': 'Disponible' if hab.disponible else 'OCUPADA',
                'Huésped': hab.huesped.nombre if hab.huesped else '-',
                'Fecha Ingreso': hab.fecha_ingreso.strftime('%d/%m/%Y') if hab.fecha_ingreso else '-'
            })
        return pd.DataFrame(data)

# INSTANCIA GLOBAL DE HOTEL
mi_hotel = Hotel()

# ==============================================================================
# 2. Interfaz Gráfica (ipywidgets)
# ==============================================================================

output_log = widgets.Output()
output_consulta = widgets.Output()
output_salida = widgets.Output()

# --- A. Pestaña: Consultar/Ocupar Habitación ---

# Formulario 1: Selección de Habitación
hab_disponibles_options = [str(i) for i in range(1, NUM_HABITACIONES + 1)]
campo_num_hab_ingreso = widgets.Dropdown(
    description='Seleccionar Habitación:',
    options=hab_disponibles_options,
    value='1',
    layout=TINY_LAYOUT
)
btn_ver_habitaciones = widgets.Button(description="Consultar Estado de Habitaciones", button_style='primary')
btn_ocupar_hab = widgets.Button(description="Continuar para Ocupar", button_style='warning')

# Contenedor para la tabla de habitaciones
tabla_habitaciones_view = widgets.Output()

# Formulario 2: Datos del Huésped
campo_fecha_ingreso = widgets.DatePicker(
    description='Fecha de Ingreso:',
    disabled=False,
    layout=WIDGET_LAYOUT
)
campo_huesped_nombre = widgets.Text(description='Nombre del Huésped:', placeholder='Nombre', layout=WIDGET_LAYOUT)
campo_huesped_apellidos = widgets.Text(description='Apellidos del Huésped:', placeholder='Apellidos', layout=WIDGET_LAYOUT)
campo_huesped_doc = widgets.Text(description='Documento de Identidad:', placeholder='Documento', layout=WIDGET_LAYOUT)

btn_registrar_ingreso = widgets.Button(description="Registrar Ingreso", button_style='success')

# Contenedor para los datos del huésped (oculto inicialmente)
form_ingreso_huesped = widgets.VBox([
    widgets.Label("### 2. Datos del Huésped y Registro de Ingreso"),
    campo_fecha_ingreso,
    campo_huesped_nombre,
    campo_huesped_apellidos,
    campo_huesped_doc,
    btn_registrar_ingreso
])

# Inicialmente ocultamos el formulario de ingreso
form_ingreso_huesped.layout.visibility = 'hidden'

# --- Lógica de la Pestaña de Ingreso ---

def actualizar_listado_habitaciones(b=None):
    """Genera y muestra la tabla de estado de habitaciones."""
    with tabla_habitaciones_view:
        clear_output(wait=True)
        df = mi_hotel.get_listado_df()
        display(HTML("<h4>Listado de Habitaciones</h4>"))
        display(df.style.set_properties(**{'font-size': '10pt'}))

def iniciar_ocupacion(b):
    """Valida la habitación seleccionada y muestra el formulario de huésped."""
    with output_log:
        clear_output(wait=True)

        try:
            num_hab = int(campo_num_hab_ingreso.value)
            habitacion = mi_hotel.get_habitacion(num_hab)

            if not habitacion:
                print("ERROR: Número de habitación inválido.")
                form_ingreso_huesped.layout.visibility = 'hidden'
                return

            if not habitacion.disponible:
                print(f"ERROR: La habitación {num_hab} ya está ocupada por: {habitacion.huesped}.")
                form_ingreso_huesped.layout.visibility = 'hidden'
                return

            print(f"CONFIRMADO: Habitación {num_hab} disponible. Proceda a ingresar los datos del huésped.")
            # Habilitar y resetear el formulario de huésped
            form_ingreso_huesped.layout.visibility = 'visible'
            campo_huesped_nombre.value = ''
            campo_huesped_apellidos.value = ''
            campo_huesped_doc.value = ''
            campo_fecha_ingreso.value = datetime.now().date()

        except ValueError:
            print("ERROR: Ingrese un número de habitación válido.")
            form_ingreso_huesped.layout.visibility = 'hidden'


def registrar_ingreso(b):
    """Registra el huésped en la habitación seleccionada y actualiza el estado."""
    with output_log:
        clear_output(wait=True)

        # Validación de campos
        nombre = campo_huesped_nombre.value.strip()
        apellidos = campo_huesped_apellidos.value.strip()
        documento = campo_huesped_doc.value.strip()
        fecha_ingreso = campo_fecha_ingreso.value

        if not nombre or not apellidos or not documento or not fecha_ingreso:
            print("ERROR: Todos los campos del huésped son obligatorios.")
            return

        try:
            num_hab = int(campo_num_hab_ingreso.value)
            habitacion = mi_hotel.get_habitacion(num_hab)

            if not habitacion or not habitacion.disponible:
                print("ERROR: La habitación ya no está disponible o es inválida.")
                return

            huesped = Huesped(nombre, apellidos, documento)
            habitacion.ocupar(huesped, fecha_ingreso)

            print(f"ÉXITO: Ingreso registrado. Habitación {num_hab} ocupada por {huesped.nombre} {huesped.apellidos}.")

            # Ocultar formulario de ingreso y actualizar listado
            form_ingreso_huesped.layout.visibility = 'hidden'
            actualizar_listado_habitaciones()

        except Exception as e:
            print(f"ERROR al registrar el ingreso: {e}")


btn_ver_habitaciones.on_click(actualizar_listado_habitaciones)
btn_ocupar_hab.on_click(iniciar_ocupacion)
btn_registrar_ingreso.on_click(registrar_ingreso)


# Contenedor final de la pestaña de Ingreso
tab_ingreso_layout = widgets.VBox([
    widgets.Label("### 1. Consulta de Habitaciones"),
    btn_ver_habitaciones,
    tabla_habitaciones_view,
    widgets.HBox([campo_num_hab_ingreso, btn_ocupar_hab]),
    form_ingreso_huesped
])


# --- B. Pestaña: Salida de Huéspedes ---

# Formulario 1: Solicitud de Habitación
campo_num_hab_salida = widgets.Text(description='Número de Habitación a Entregar:', placeholder='1-10', layout=TINY_LAYOUT)
btn_buscar_hab_salida = widgets.Button(description="Buscar Habitación", button_style='primary')

# Contenedor de la información de salida (oculto inicialmente)
info_salida_display = widgets.Output()

# Formulario 2: Registro de Salida
campo_fecha_salida = widgets.DatePicker(
    description='Fecha de Salida:',
    disabled=False,
    layout=WIDGET_LAYOUT
)

btn_calcular_total = widgets.Button(description="Calcular Total a Pagar", button_style='info', disabled=True)
btn_registrar_salida = widgets.Button(description="Registrar Salida y Liberar Habitación", button_style='success', disabled=True)

# Variables de estado para la salida
habitacion_actual = None # Almacena el objeto Habitacion
total_a_pagar_label = widgets.HTML(value="")

form_salida_registro = widgets.VBox([
    widgets.Label("### 2. Registro de Salida"),
    info_salida_display,
    campo_fecha_salida,
    widgets.HBox([btn_calcular_total, btn_registrar_salida]),
    total_a_pagar_label
])

form_salida_registro.layout.visibility = 'hidden'

# --- Lógica de la Pestaña de Salida ---

def buscar_habitacion_salida(b):
    """Busca la habitación y valida si está ocupada para la salida."""
    global habitacion_actual

    with output_log:
        clear_output(wait=True)
        info_salida_display.clear_output()
        form_salida_registro.layout.visibility = 'hidden'
        btn_calcular_total.disabled = True
        btn_registrar_salida.disabled = True
        total_a_pagar_label.value = ""

        try:
            num_hab = int(campo_num_hab_salida.value.strip())
            habitacion = mi_hotel.get_habitacion(num_hab)

            if not habitacion:
                print("ERROR: Número de habitación inválido (debe ser 1 a 10).")
                return

            if habitacion.disponible:
                print(f"ERROR: La habitación {num_hab} no está ocupada.")
                return

            # Habitación encontrada y ocupada
            habitacion_actual = habitacion

            with info_salida_display:
                clear_output(wait=True)
                display(HTML(f"<b>Habitación:</b> {habitacion_actual.numero} (Precio/Día: ${habitacion_actual.precio_dia:,.0f})"))
                display(HTML(f"<b>Huésped:</b> {habitacion_actual.huesped.nombre} {habitacion_actual.huesped.apellidos}"))
                display(HTML(f"<b>Fecha Ingreso:</b> {habitacion_actual.fecha_ingreso.strftime('%d/%m/%Y')}"))

            # Habilitar el formulario de registro de salida
            form_salida_registro.layout.visibility = 'visible'
            campo_fecha_salida.value = datetime.now().date()


            btn_calcular_total.disabled = False

            print(f"CONFIRMADO: Habitación {num_hab} lista para el registro de salida. Ingrese la fecha de salida y calcule el total.")

        except ValueError:
            print("ERROR: Ingrese un número válido para la habitación.")
        except Exception as e:
            print(f"ERROR inesperado: {e}")

def calcular_total_pagar(b):
    """Calcula los días de alojamiento y el costo total."""
    if not habitacion_actual:
        total_a_pagar_label.value = "<b style='color: red;'>ERROR: Seleccione una habitación primero.</b>"
        return

    fecha_ingreso = habitacion_actual.fecha_ingreso
    fecha_salida = campo_fecha_salida.value

    with output_log:
        clear_output(wait=True)
        total_a_pagar_label.value = ""
        btn_registrar_salida.disabled = True

        if not fecha_salida:
            print("ERROR: Debe seleccionar una fecha de salida.")
            return

        if fecha_salida <= fecha_ingreso:
            print("ERROR: La fecha de salida debe ser posterior a la fecha de ingreso.")
            return

        # Cálculo
        dias_alojamiento = (fecha_salida - fecha_ingreso).days
        total_pagar = dias_alojamiento * habitacion_actual.precio_dia

        # Mostrar resultado
        total_a_pagar_label.value = f"""
            <div style='border: 1px solid #007bff; padding: 10px; border-radius: 5px; background-color: #f0f8ff;'>
                <p><b>Días de Alojamiento:</b> {dias_alojamiento} días</p>
                <h4 style='color: #28a745;'>TOTAL A PAGAR: ${total_pagar:,.2f}</h4>
            </div>
        """
        btn_registrar_salida.disabled = False
        print("Cálculo realizado. Puede proceder a registrar la salida.")


def registrar_salida_action(b):
    """Libera la habitación y resetea el estado."""
    global habitacion_actual

    with output_log:
        clear_output(wait=True)

        if not habitacion_actual:
            print("ERROR: No hay habitación seleccionada para la salida.")
            return

        # Liberar la habitación
        num = habitacion_actual.numero
        habitacion_actual.liberar()

        print(f"ÉXITO: Se ha registrado la salida del huésped. La Habitación {num} ahora está DISPONIBLE.")

        # Resetear la interfaz de salida
        campo_num_hab_salida.value = ''
        form_salida_registro.layout.visibility = 'hidden'
        habitacion_actual = None
        total_a_pagar_label.value = ""
        btn_calcular_total.disabled = True
        btn_registrar_salida.disabled = True

        actualizar_listado_habitaciones()

btn_buscar_hab_salida.on_click(buscar_habitacion_salida)
btn_calcular_total.on_click(calcular_total_pagar)
btn_registrar_salida.on_click(registrar_salida_action)

# Contenedor final de la pestaña de Salida
tab_salida_layout = widgets.VBox([
    widgets.Label("### 1. Búsqueda de Habitación"),
    widgets.HBox([campo_num_hab_salida, btn_buscar_hab_salida]),
    form_salida_registro
])


# ==============================================================================
# 3. Estructura y Display de la Aplicación Principal
# ==============================================================================

# Se crea un contenedor de pestañas para las dos opciones de menú
tab_widget = widgets.Tab()
tab_widget.children = [
    tab_ingreso_layout,
    tab_salida_layout
]

# Títulos de las pestañas
tab_widget.set_title(0, 'Consultar/Ocupar Habitación')
tab_widget.set_title(1, 'Salida de Huéspedes')

# Título Principal
titulo = widgets.HTML("<h2>Gestión de Hotel (10 Habitaciones)</h2>")

# Diseño final de la aplicación
app_layout = widgets.VBox([
    titulo,
    tab_widget,
    widgets.HTML("<hr>"),
    widgets.Label("Log de Operaciones y Mensajes:"),
    output_log
])

# Mostrar la interfaz en Colab
display(app_layout)

## Ejercicio 9.1 página 589
## Escenarios

In [ ]:
# @title
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
from datetime import date


# 1. Clases POO


class Contacto:
    """Clase que define un contacto para una agenda personal."""
    def __init__(self, nombres, apellidos, fecha_nacimiento, direccion, telefono, correo):
        self.nombres = nombres
        self.apellidos = apellidos
        self.fecha_nacimiento = fecha_nacimiento
        self.direccion = direccion
        self.telefono = telefono
        self.correo = correo

    def to_simple_html(self):
        """Retorna una representación formateada y condensada del contacto para la lista."""
        fecha_str = self.fecha_nacimiento.strftime("%d/%m/%Y") if self.fecha_nacimiento else "N/A"


        return f"""
        <div style="margin-bottom: 10px; padding: 10px; border: 1px solid #ddd; border-radius: 5px; background-color: white;">
            <p style="margin: 0; font-weight: bold;"><i class="fas fa-user"></i> {self.nombres} {self.apellidos}</p>
            <p style="margin: 5px 0 0 0; font-size: small;">
                <i class="fas fa-calendar-alt"></i> {fecha_str} |
                <i class="fas fa-map-marker-alt"></i> {self.direccion} |
                <i class="fas fa-phone"></i> {self.telefono} |
                <i class="fas fa-envelope"></i> {self.correo}
            </p>
        </div>
        """

class ListaContactos:
    """Clase que define una lista de objetos de tipo Contacto."""
    def __init__(self):
        self.lista = [] # Lista de Python para almacenar objetos Contacto

    def agregar_contacto(self, contacto):
        """Método que agrega un contacto a la lista."""
        self.lista.append(contacto)

# INSTANCIA GLOBAL DE LA LISTA DE CONTACTOS
agenda = ListaContactos()

# ==============================================================================
# 2. Interfaz Gráfica (ipywidgets)
# ==============================================================================

# Layout y Estilos
GRID_ITEM_LAYOUT = widgets.Layout(width='auto', padding='5px') # Para GridBox
FIELD_LAYOUT = widgets.Layout(width='280px')
LIST_LAYOUT = widgets.Layout(width='550px', height='300px', overflow='auto', background_color='white')
OUTPUT_LAYOUT = widgets.Layout(height='40px')

# Widgets de Entrada
# Etiquetas
label_nombres = widgets.Label('Nombres:', layout=GRID_ITEM_LAYOUT)
label_apellidos = widgets.Label('Apellidos:', layout=GRID_ITEM_LAYOUT)
label_fecha_nacimiento = widgets.Label('Fecha Nacimiento:', layout=GRID_ITEM_LAYOUT)
label_direccion = widgets.Label('Dirección:', layout=GRID_ITEM_LAYOUT)
label_telefono = widgets.Label('Teléfono:', layout=GRID_ITEM_LAYOUT)
label_correo = widgets.Label('Correo Electrónico:', layout=GRID_ITEM_LAYOUT)

# Campos de texto/fecha
campo_nombres = widgets.Text(layout=FIELD_LAYOUT)
campo_apellidos = widgets.Text(layout=FIELD_LAYOUT)
campo_fecha_nacimiento = widgets.DatePicker(value=None, layout=FIELD_LAYOUT) # DatePicker simula el calendario
campo_direccion = widgets.Text(layout=FIELD_LAYOUT)
campo_telefono = widgets.Text(layout=FIELD_LAYOUT)
campo_correo = widgets.Text(layout=FIELD_LAYOUT)

# Botón y Salida de Mensajes
btn_agregar = widgets.Button(description="Agregar", button_style='success', layout=widgets.Layout(width='85%'))
output_alertas = widgets.Output(layout=OUTPUT_LAYOUT)

# Usamos HTML para un fondo blanco y control de formato
lista_visual = widgets.HTML(
    value='<p style="color:#666;">Lista de contactos agregados: (Vacía)</p>',
    layout=LIST_LAYOUT
)

icon_style = widgets.HTML(
    value='<link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/font-awesome/5.15.4/css/all.min.css">'
)


# ==============================================================================
# 3. Lógica de la Aplicación
# ==============================================================================

def limpiar_campos():
    """Limpia todos los campos del formulario."""
    campo_nombres.value = ""
    campo_apellidos.value = ""
    campo_fecha_nacimiento.value = None
    campo_direccion.value = ""
    campo_telefono.value = ""
    campo_correo.value = ""

def actualizar_lista_visual():
    """Actualiza el widget HTML con los contactos de la lista."""
    if not agenda.lista:
        lista_visual.value = '<p style="color:#666;">Lista de contactos agregados: (Vacía)</p>'
        return

    contenido = ""
    # Recorremos la lista en reversa para que el contacto más reciente aparezca primero
    for contacto in reversed(agenda.lista):
        contenido += contacto.to_simple_html()

    lista_visual.value = contenido

def agregar_contacto(b):
    """
    Captura los datos, valida, crea un objeto Contacto y lo añade a la lista.
    """
    with output_alertas:
        clear_output(wait=True)

        # 1. Capturar y validar datos
        nombres = campo_nombres.value.strip()
        apellidos = campo_apellidos.value.strip()
        fecha_nacimiento = campo_fecha_nacimiento.value # Es un objeto date o None
        direccion = campo_direccion.value.strip()
        telefono = campo_telefono.value.strip()
        correo = campo_correo.value.strip()

        # Validar campos obligatorios
        campos_vacios = [
            ("Nombres", nombres),
            ("Apellidos", apellidos),
            ("Dirección", direccion),
            ("Teléfono", telefono),
            ("Correo Electrónico", correo)
        ]

        vacios = [nombre for nombre, valor in campos_vacios if not valor]

        if vacios:
            print(f"ERROR: No se permiten campos vacíos. Faltan: {', '.join(vacios)}.")
            return

        # 2. Si los datos son correctos
        nuevo_contacto = Contacto(nombres, apellidos, fecha_nacimiento, direccion, telefono, correo)
        agenda.agregar_contacto(nuevo_contacto)

        # Mostrar mensaje de confirmación
        print(f"ÉXITO: Contacto de {nombres} {apellidos} agregado.")

        # Actualizar la vista de la lista y limpiar campos
        actualizar_lista_visual()
        limpiar_campos()

# Asignar la función al evento click del botón
btn_agregar.on_click(agregar_contacto)

# ==============================================================================
# 4. Diseño del Layout
# ==============================================================================

# 4.1. Formulario de Entrada (Izquierda) -
grid_contenido = widgets.GridBox(
    children=[
        label_nombres, campo_nombres,
        label_apellidos, campo_apellidos,
        label_fecha_nacimiento, campo_fecha_nacimiento,
        label_direccion, campo_direccion,
        label_telefono, campo_telefono,
        label_correo, campo_correo,
    ],
    layout=widgets.Layout(
        width='100%',
        grid_template_columns='minmax(120px, max-content) 1fr', # Columna 1: etiquetas, Columna 2: campos
        grid_gap='5px'
    )
)

form_entrada = widgets.VBox([
    widgets.HTML("<h3>Ingreso de Contacto</h3>"),
    grid_contenido,
    widgets.HBox([widgets.Label("", layout=widgets.Layout(width='125px')), btn_agregar]), # Alinea el botón bajo el campo
    widgets.HTML("<hr>"),
    widgets.Label("Mensajes / Alertas:"),
    output_alertas
], layout=widgets.Layout(border='2px solid #007bff', padding='15px', width='450px', min_width='400px'))

# 4.2. Lista de Contactos (Derecha)
lista_output = widgets.VBox([
    widgets.HTML("<h3>Lista de Contactos</h3>"),
    lista_visual
], layout=widgets.Layout(
    border='2px solid #28a745',
    padding='15px',
    width='550px',
    min_width='500px',
    height='470px' # Ajustamos la altura para alinearse con el formulario
))


# Contenedor principal
contenedor_principal = widgets.HBox([form_entrada, lista_output], layout=widgets.Layout(align_items='stretch'))

# Título de la aplicación
titulo = widgets.HTML("<h2>Agenda Personal de Contactos</h2>")

# Diseño final de la aplicación
app_layout = widgets.VBox([
    icon_style, # Importa Font Awesome
    titulo,
    contenedor_principal
])

# Mostrar la interfaz
display(app_layout)